In [10]:
import requests
import json
from Crypto.PublicKey import RSA
import time
import datetime

from bs4 import BeautifulSoup

from util.crypto import encrypt_data
from util.html_tools import get_variable_from_html

In [3]:
def create_steam_auth_session():
    session = requests.Session()
    response = session.get("https://steamcommunity.com/login/home/?goto=")
    user = input("Steam Username:")
    password = input("Password:")
    login_params = {
        "username": user
    }

    response = session.post("https://steamcommunity.com/login/home/getrsakey/",
        params = login_params)
    responseJSON = json.loads(response.text)
    
    
    exp = int(responseJSON["publickey_exp"],16)
    mod = int(responseJSON["publickey_mod"],16)
    
    rsa_key = RSA.construct((mod, exp))
    
    encrypted_password = encrypt_data(password, rsa_key)
    
    twofactorcode = ""
    m_gidCaptcha = "-1"
    captchaText = ""
    m_steamidEmailAuth = ""
    m_unRequestedTokenType = "-1"
    login_params = {
        "donotcache":round(time.time() * 1000),
        "password":encrypted_password,
        "username":user,
        "twofactorcode": twofactorcode,
        "emailauth":"",
        "loginfriendlyname":"",
        "captchagid":m_gidCaptcha,
        "captcha_text":captchaText,
        "emailsteamid":m_steamidEmailAuth,
        "rsatimestamp":responseJSON["timestamp"],
        "remember_login": 'false',
        "tokentype": m_unRequestedTokenType
        
    }
    
    header = {
        "Content-Type":"application/x-www-form-urlencoded; charset=UTF-8",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
    }
    
    
    response = session.post("https://steamcommunity.com/login/home/dologin",
                     data = login_params
                     )
    responseJSON = json.loads(response.text)

    if responseJSON["success"] == True:
        print(f"Successfully logged in {user}")
        return s

    require_twofactor = False
    require_email = False
    if "requires_twofactor" in responseJSON:
        if responseJSON["requires_twofactor"] == True:
            require_twofactor = True
    if "emailauth_needed" in responseJSON:
        if responseJSON["emailauth_needed"] == True:
            require_email = True

    if not require_email and not require_twofactor:
        print("Session could not be established, wrong password or username")
        return None
    
    if require_twofactor:
        twofactorcode = input("Enter your 2FA code:")
        login_params["twofactorcode"] = twofactorcode
    else:
        email_code = input("Enter your email confirmation code:")
        login_params["emailauth"] = email_code

    response = session.post("https://steamcommunity.com/login/home/dologin",
                 data = login_params
                 )
    responseJSON = json.loads(response.text)
    if responseJSON["success"] == True:
        return session
    else:
        print("Login failed")
        print(responseJSON)
    
        

In [9]:
def create_inventory_history_list(soup):
    history_list = []
    rows = soup.find_all("div", "tradehistoryrow")
    for row in rows:
        history_element = {
            "description": "",
            "timestamp": None,
            "new_items": [],
            "lost_items": [],
        }
        date = row.find_all("div", "tradehistory_date")[0].text.strip().split("\t")
        date = [x for x in date if x]
        day = int(date[0].split(" ")[0])
        month = date[0].split(" ")[1][:-1]
        year = int(date[0].split(" ")[-1])
        hour = int(date[1].split(":")[0])
        minute = int(date[1].split(":")[1][:-2])
        ampm = date[1][-2:]
        history_element["timestamp"] = time.mktime(datetime.datetime.strptime(f"{day:02d}/{month}/{year} {hour:02d}:{minute:02d} {ampm}", "%d/%b/%Y %H:%M %p").timetuple())
        history_element["description"] = row.find_all("div", "tradehistory_event_description")[0].text.strip()
        item_changes = row.find_all("div", "tradehistory_items tradehistory_items_withimages")
        for item_change in item_changes:
            item_type = item_change.find("div", "tradehistory_plusminus").text.strip()
            print(item_type)
            print(item_change)
        
        history_list.append(history_element)
    pass

In [13]:
def get_inventory_history(session):
    response = session.get("https://steamcommunity.com/")
    if response.status_code != 200:
        return None
    steamid = get_variable_from_html("g_steamID", response.text)
    response = session.get(f"https://steamcommunity.com/profiles/{steamid}/inventoryhistory/?app[]=730")
    if response.status_code != 200:
        return None
    profile_link = get_variable_from_html("g_strProfileURL", response.text).replace("\\", "")
    sessionid = get_variable_from_html("g_sessionID", response.text)
    filter_apps = get_variable_from_html("g_rgFilterApps", response.text)


    cursor = get_variable_from_html("g_historyCursor", response.text)
    #item_json = get_variable_from_html("g_rgDescriptions", response.text)

    full_site = response.text
    soup = BeautifulSoup(full_site, 'html.parser')
    return soup#, item_json

    while True:
        return
        req_data = {
            "ajax": "1",
            "sessionid": sessionid,
        }
        for key in cursor:
            req_data[f"cursor[{key}]"] = cursor[key]
        for num, app in enumerate(filter_apps):
            req_data[f"app[{num}]"] = app
        response = session.get(profile_link+"/inventoryhistory/", params=req_data)
        response_JSON = json.loads(response.text)
        if cursor in response_JSON.keys():
            cursor = response_JSON["cursor"]
        else:
            break  
    return full_site

In [4]:
session = create_steam_auth_session()

In [14]:
soup, items = get_inventory_history(session)

SyntaxError: EOL while scanning string literal (<unknown>, line 1)

In [12]:
soup

<!DOCTYPE html>

<html class="responsive" lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="#171a21" name="theme-color"/>
<title>Steam Community :: Assassino :: Inventory History</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://community.akamai.steamstatic.com/public/shared/css/motiva_sans.css?v=-DH0xTYpnVe2&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://community.akamai.steamstatic.com/public/shared/css/buttons.css?v=n-eRNszNIRMH&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://community.akamai.steamstatic.com/public/shared/css/shared_global.css?v=29MQRF5DS62e&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://community.akamai.steamstatic.com/public/css/globalv2.css?v=Tj_Gb074U72O&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://communit